In [1]:
from __future__ import print_function
import os
# COM-Server
import win32com.client as com

# Vissim = com.gencache.EnsureDispatch("Vissim.Vissim")
Vissim = com.Dispatch("Vissim.Vissim")
# Vissim.New()

Path_of_COM_Basic_Commands_network = os.getcwd()


Filename = os.path.join(Path_of_COM_Basic_Commands_network, 'kia_parking_8.inpx')

flag_read_additionally = False # you can read network(elements) additionally, in this case set "flag_read_additionally" to true
Vissim.LoadNet(Filename, flag_read_additionally)

In [190]:
parking_lots = {
    "uber_go": {
        "decisions": [94, 95, 96, 97, 98, 99, 100, 101, 102, 103],
        "stage": [94, 95],
        "lot_numbers": list(range(2581, 2591)),
        "lot_1" : 2583,
        "route_start": 38,
        "route_end": 46
    },
    "uber_xl": {
        "decisions": [143, 144],
        "stage": [143],
        "lot_numbers": list(range(5998, 6001)),
        "lot_1" : 5999,
        "route_start": 37,
        "route_end": 37
    },
    "blusmart": {
        "decisions": [132, 133, 134, 135, 136, 137],
        "stage": [132],
        "lot_numbers": list(range(9000, 9007)),
        'lot_1' : 9001,
        "route_start": 36,
        "route_end": 36
    },
    "ola_mini": {
        "decisions": [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130],
        "stage": [120],
        "lot_numbers": list(range(5871, 5883)),
        "lot_1" : 5872,
        "route_start": 29,
        "route_end": 34
    },
    "ola_suv": {
        "decisions": [85],
        "stage": [85],
        "lot_numbers": list(range(6999, 7001)),
        "lot_1" : 7000,
        "route_start": 35,
        "route_end": 35
    },
    "airport_taxi": {
        "decisions": [198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209],
        "stage": [200, 199, 198],
        "lot_numbers": list(range(7997, 8008)),
        "lot_1" : 8000,
        "route_start": 58,
        "route_end": 84
    }
}


assigned = [-1 for _ in range(10000)]


In [185]:
def assign_parking_lot(lot_numbers, vehicle_info, assigned, parking_lots_vissim, vehicles):
    for lot_no in lot_numbers:
        lot = parking_lots_vissim.ItemByKey(lot_no)
        if lot.AttValue('CurAvail') == 0:
            assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
        if assigned[lot_no] and assigned[lot_no] != -1:
            vehicle = vehicles.ItemByKey(assigned[lot_no])
            if lot_no != vehicle.AttValue('DestParkLot'):
                assigned[lot.AttValue('No')] = -1

In [186]:
def find_park_duration_init(vehicle_list, parking_lots):
    vehicle_info = {
        85: {'no': 0, 'park_dur': 0},
        94: {'no': 0, 'park_dur': 0},
        95: {'no': 0, 'park_dur': 0},
        120: {'no': 0, 'park_dur': 0},
        132: {'no': 0, 'park_dur': 0},
        143: {'no': 0, 'park_dur': 0},
        200: {'no': 0, 'park_dur': 0},
        199: {'no': 0, 'park_dur': 0},
        198: {'no': 0, 'park_dur': 0}
    }

    for veh in vehicle_list:
        dest_lot = veh.AttValue('DestParkLot')
        rout_dec_no = veh.AttValue('RoutDecNo')
        park_state = veh.AttValue('ParkState')
        if dest_lot is None or park_state != 'ISPARKED':
            continue

        dest_lot = int(dest_lot)
        rout_dec_no = int(rout_dec_no)


        for lot_type, lot_info in parking_lots.items():
            # print(f"Checking for {lot_type}")
            # print(f"Dest lot: {dest_lot}, Rout dec no: {rout_dec_no}")
            if dest_lot in lot_info["lot_numbers"] and rout_dec_no in lot_info["decisions"]:
                next_dest = lot_info["lot_1"] if rout_dec_no in lot_info["stage"] else dest_lot + 1
                # print(f"Next dest: {next_dest}")
                if Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] is None or assigned[next_dest] == -1):
                    if rout_dec_no in [94, 95, 198, 199, 200]:
                        vehicle_info[rout_dec_no]['no'] = veh.AttValue('No')
                        vehicle_info[rout_dec_no]['park_dur'] = veh.AttValue('ParkDurCur')
                    else:
                        assigned[next_dest] = veh.AttValue('No')
                        # Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)
                        veh.SetAttValue('DwellTm', 0)
                break
                

    return vehicle_info


In [170]:
Vissim.SuspendUpdateGUI()

In [191]:
for i in range(6000):
    
    Vissim.Simulation.RunSingleStep()
    
    # VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking
    
    # Run the following code every 100 simulation steps
    if i % 50 == 0:
        vehicles = Vissim.Net.Vehicles
        # Calculate vehicle park duration information
        vehicle_info = find_park_duration_init(vehicles, parking_lots)
        parking_lots_vissim = Vissim.Net.ParkingLots
        
        # Update dwell time based on park duration
        # Update dwell time based on maximum park duration for keys 94 and 95
        max_park_dur_94_95 = max(vehicle_info[94]['park_dur'], vehicle_info[95]['park_dur'])
        for lot in [94, 95]:
            if vehicle_info[lot]['park_dur'] == max_park_dur_94_95 and max_park_dur_94_95 > 0:
                Vissim.Net.Vehicles.ItemByKey(vehicle_info[lot]['no']).SetAttValue('DwellTm', 0)
                assigned[2583] = vehicle_info[lot]['no']
                break
        
        # Update dwell time based on maximum park duration for keys 198, 199, 200
        max_park_dur_198_199_200 = max(vehicle_info[198]['park_dur'], vehicle_info[199]['park_dur'], vehicle_info[200]['park_dur'])
        for lot in [200, 199, 198]:
            if vehicle_info[lot]['park_dur'] == max_park_dur_198_199_200 and max_park_dur_198_199_200 > 0:
                Vissim.Net.Vehicles.ItemByKey(vehicle_info[lot]['no']).SetAttValue('DwellTm', 0)
                assigned[8000] = vehicle_info[lot]['no']
                break
        
        # Assign parking lots
        for service, lots_info in parking_lots.items():
            assign_parking_lot(lots_info['lot_numbers'], vehicle_info, assigned, parking_lots_vissim, vehicles)

# return  time_vissim
        

    

In [177]:
Vissim.ResumeUpdateGUI(True)

# Main Simulation

In [181]:
# Define pedestrian areas and their corresponding vehicle lots
pedes_area_to_lot = {
    2590 : 51,
    6000 : 52,
    9006 : 50,
    5882 : 22,
    7000 : 29,
    8007 : 37
}

# Initialize counts for each vehicle lot
count_veh = {lot: 0 for lot in pedes_area_to_lot.keys()}

# Initialize previous passenger counts for each pedestrian area
prev_passenger_in_queue = {area: 0 for area in pedes_area_to_lot.values()}

# Initialize reserved vehicles for each vehicle lot
reserved_vehs = {lot: {'no': 0, 'park_dur': 0} for lot in pedes_area_to_lot.keys()}


In [166]:
def find_park_duration(vehicle_list, parking_lots, parking_lots_vissim):
    
    vehicle_info = {
        85: {'no': 0, 'park_dur': 0},
        94: {'no': 0, 'park_dur': 0},
        95: {'no': 0, 'park_dur': 0},
        120: {'no': 0, 'park_dur': 0},
        132: {'no': 0, 'park_dur': 0},
        143: {'no': 0, 'park_dur': 0},
        200: {'no': 0, 'park_dur': 0},
        199: {'no': 0, 'park_dur': 0},
        198: {'no': 0, 'park_dur': 0}
    }
    reserved_vehs = {lot: {'no': 0, 'park_dur': 0} for lot in pedes_area_to_lot.keys()}
    # parking_lots_vissim = Vissim.Net.ParkingLots

    for veh in vehicle_list:
        dest_lot = veh.AttValue('DestParkLot')
        rout_dec_no = veh.AttValue('RoutDecNo')
        park_state = veh.AttValue('ParkState')
        if dest_lot is None or park_state != 'ISPARKED':
            continue

        dest_lot = int(dest_lot)
        rout_dec_no = int(rout_dec_no)
        park_dur = int(veh.AttValue('ParkDurCur'))

        for lot_type, lot_info in parking_lots.items():
            if rout_dec_no >= lot_info["route_start"] and rout_dec_no <= lot_info["route_end"]:
                if park_dur > reserved_vehs[lot_info['lot_numbers'][-1]]['park_dur']:
                    reserved_vehs[lot_info['lot_numbers'][-1]]['no'] = veh.AttValue('No')
                    reserved_vehs[lot_info['lot_numbers'][-1]]['park_dur'] = park_dur
                break
                
        for lot_type, lot_info in parking_lots.items():
            # print(f"Checking for {lot_type}")
            # print(f"Dest lot: {dest_lot}, Rout dec no: {rout_dec_no}")
            if dest_lot in lot_info["lot_numbers"] and rout_dec_no in lot_info["decisions"]:
                next_dest = lot_info["lot_1"] if rout_dec_no in lot_info["stage"] else dest_lot + 1
                # print(f"Next dest: {next_dest}")
                if parking_lots_vissim.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] is None or assigned[next_dest] == -1):
                    if rout_dec_no in [94, 95, 198, 199, 200]:
                        vehicle_info[rout_dec_no]['no'] = veh.AttValue('No')
                        vehicle_info[rout_dec_no]['park_dur'] = veh.AttValue('ParkDurCur')
                    else:
                        assigned[next_dest] = veh.AttValue('No')
                        # Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)
                        veh.SetAttValue('DwellTm', 0)
                break
                

    return vehicle_info,reserved_vehs 

In [167]:
def passenger_values(pedestrian):
    passenger = {51: {'no': None, 'pos': None}, 52: {'no': None, 'pos': None}, 50: {'no': None, 'pos': None}, 22: {'no': None, 'pos': None}, 29: {'no': None, 'pos': None}, 37: {'no': None, 'pos': None}}
    
    for pedes in pedestrian:
        for veh_lot, pedes_area in pedes_area_to_lot.items():
            if pedes.AttValue('ConstrElNo') == pedes_area and pedes.AttValue('PosInQueue') == 1 and pedes.AttValue("MotionState") == 'WAITINGATQUEUEHEAD':
                passenger[pedes_area]['no'] = pedes.AttValue('No') 
                passenger[pedes_area]['pos'] = pedes.AttValue('PosInQueue')
                break
                
    return passenger

In [192]:
Vissim.SuspendUpdateGUI()

In [182]:
for i in range(864000):
    
    Vissim.Simulation.RunSingleStep()
    # Vissim.SuspendUpdateGUI()
    
    
    # Run the following code every 100 simulation steps
    if i % 50 == 0:
        pedestrian = Vissim.Net.Pedestrians
        vehicles = Vissim.Net.Vehicles
        parking_lots_vissim = Vissim.Net.ParkingLots
        # VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking
        # Calculate vehicle park duration information
        vehicle_info, reserved_vehs = find_park_duration(vehicles, parking_lots, parking_lots_vissim)
        
        # Update dwell time based on park duration
        # Update dwell time based on maximum park duration for keys 94 and 95
        max_park_dur_94_95 = max(vehicle_info[94]['park_dur'], vehicle_info[95]['park_dur'])
        for lot in [94, 95]:
            if vehicle_info[lot]['park_dur'] == max_park_dur_94_95 and max_park_dur_94_95 > 0:
                vehicles.ItemByKey(vehicle_info[lot]['no']).SetAttValue('DwellTm', 0)
                assigned[2583] = vehicle_info[lot]['no']
                break
        
        # Update dwell time based on maximum park duration for keys 198, 199, 200
        max_park_dur_198_199_200 = max(vehicle_info[198]['park_dur'], vehicle_info[199]['park_dur'], vehicle_info[200]['park_dur'])
        for lot in [200, 199, 198]:
            if vehicle_info[lot]['park_dur'] == max_park_dur_198_199_200 and max_park_dur_198_199_200 > 0:
                vehicles.ItemByKey(vehicle_info[lot]['no']).SetAttValue('DwellTm', 0)
                assigned[8000] = vehicle_info[lot]['no']
                break
        
        # Assign parking lots
        for service, lots_info in parking_lots.items():
            assign_parking_lot(lots_info['lot_numbers'], vehicle_info, assigned, parking_lots_vissim, vehicles)

        # Update pedestrian values
        passenger = passenger_values(pedestrian)

        for veh_lot, pedes_area in pedes_area_to_lot.items():
            pedes_no = passenger[pedes_area]['no'] 
            pedes_pos = passenger[pedes_area]['pos']

            veh_no = parking_lots_vissim.ItemByKey(veh_lot).AttValue(r'Last:VehParking\No')

            if veh_no and pedes_no and pedes_pos == 1:
                vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', 0)
                pedestrian.ItemByKey(pedes_no).SetAttValue('DwellTm', 0)

                count_veh[veh_lot] += 1
                prev_passenger_in_queue[pedes_area] = pedes_no

        for veh_lot, count in count_veh.items():
            if count > 0 and reserved_vehs[veh_lot]['no'] != 0:
                vehicles.ItemByKey(reserved_vehs[veh_lot]['no']).SetAttValue('DwellTm', 0)
                reserved_vehs[veh_lot]['no'] = 0
                reserved_vehs[veh_lot]['park_dur'] = 0
                count_veh[veh_lot] -= 1
